In [1]:
from brian2 import *
prefs.codegen.target = 'cppyy'
prefs.codegen.runtime.cppyy.enable_introspection = True

In [2]:
import cppyy # check import

In [3]:
# --- Build a simple network ---
n = 100
duration = 100 * ms
tau = 10 * ms

eqs = '''
dv/dt = (v0 - v) / tau : volt (unless refractory)
v0 : volt
'''

group = NeuronGroup(n, eqs, threshold='v > 10*mV', reset='v = 0*mV',
                    refractory=5 * ms, method='exact')
group.v = 0 * mV
group.v0 = '20*mV * i / (n-1)'

# Add a monitor so we can compare before/after
mon = StateMonitor(group, 'v', record=[0, 50, 99])

# --- Run the simulation (this triggers compilation) ---
run(duration)

In [4]:
# --- Now use the introspector ---
from brian2.codegen.runtime.cppyy_rt.introspector import get_introspector

intro = get_introspector()
assert intro is not None, "Introspector should be active!"


In [5]:
# ---- 1. List all compiled code objects ----
intro.list_objects()

Code Object,Template,Compiled Blocks,# Variables
neurongroup_spike_resetter_codeobject,reset,run,5
neurongroup_spike_thresholder_codeobject,threshold,run,8
neurongroup_stateupdater_codeobject,stateupdate,run,11
statemonitor_codeobject,statemonitor,run,8


In [6]:
# ---- 2. Inspect the state updater ----
# Using glob pattern — "stateupdater*" matches the full name
intro.inspect("*stateupdater*")

#,C++ Type,Parameter Name,Namespace Key,Current Value
0,int64_t,N,N,int = 100
1,double*,_ptr_array_defaultclock_dt,_ptr_array_defaultclock_dt,"ndarray((1,), float64) = [0.0001]"
2,double*,_ptr_array_neurongroup_lastspike,_ptr_array_neurongroup_lastspike,"ndarray((100,), float64) range=[-1e+04, -1e+04]"
3,int,_numlastspike,_numlastspike,int = 100
4,int8_t*,_ptr_array_neurongroup_not_refractory,_ptr_array_neurongroup_not_refractory,"ndarray((100,), bool) range=[1, 1]"
5,int,_numnot_refractory,_numnot_refractory,int = 100
6,double*,_ptr_array_defaultclock_t,_ptr_array_defaultclock_t,"ndarray((1,), float64) = [0.1]"
7,double,tau,tau,float = 0.01
8,double*,_ptr_array_neurongroup_v,_ptr_array_neurongroup_v,"ndarray((100,), float64) range=[0, 0.009957]"
9,int,_numv,_numv,int = 100


In [7]:
# ---- 3. View just the params ----
intro.params("*stateupdater*")

#,C++ Type,Parameter Name,Namespace Key,Current Value
0,int64_t,N,N,int = 100
1,double*,_ptr_array_defaultclock_dt,_ptr_array_defaultclock_dt,"ndarray((1,), float64) = [0.0001]"
2,double*,_ptr_array_neurongroup_lastspike,_ptr_array_neurongroup_lastspike,"ndarray((100,), float64) range=[-1e+04, -1e+04]"
3,int,_numlastspike,_numlastspike,int = 100
4,int8_t*,_ptr_array_neurongroup_not_refractory,_ptr_array_neurongroup_not_refractory,"ndarray((100,), bool) range=[1, 1]"
5,int,_numnot_refractory,_numnot_refractory,int = 100
6,double*,_ptr_array_defaultclock_t,_ptr_array_defaultclock_t,"ndarray((1,), float64) = [0.1]"
7,double,tau,tau,float = 0.01
8,double*,_ptr_array_neurongroup_v,_ptr_array_neurongroup_v,"ndarray((100,), float64) range=[0, 0.009957]"
9,int,_numv,_numv,int = 100


In [8]:
# ---- 4. View the namespace ----
intro.namespace("*stateupdater*")

Key,Value
_ptr_array_defaultclock_dt,"ndarray((1,), float64) = [0.0001]"
_ptr_array_defaultclock_t,"ndarray((1,), float64) = [0.1]"
_ptr_array_neurongroup_lastspike,"ndarray((100,), float64) range=[-1e+04, -1e+04]"
_ptr_array_neurongroup_not_refractory,"ndarray((100,), bool) range=[1, 1]"
_ptr_array_neurongroup_v,"ndarray((100,), float64) range=[0, 0.009957]"
_ptr_array_neurongroup_v0,"ndarray((100,), float64) range=[0, 0.02]"
Key,Value
_numdt,int = 1
_numlastspike,int = 100
_numnot_refractory,int = 100


In [9]:
# ---- 5. View C++ globals ----
intro.cpp_globals()

['_brian_cppyy_rng',
 '_brian_cppyy_run_neurongroup_spike_resetter_codeobject',
 '_brian_cppyy_run_neurongroup_spike_thresholder_codeobject',
 '_brian_cppyy_run_neurongroup_stateupdater_codeobject',
 '_brian_cppyy_run_statemonitor_codeobject']

In [10]:
# ---- 6. Evaluate a C++ expression ----
print("\n" + "=" * 60)
print("EVAL C++")
print("=" * 60)
print(f"M_PI = {intro.eval_cpp('M_PI')}")
print(f"sizeof(double) = {intro.eval_cpp('sizeof(double)', 'size_t')}")
print(f"_brian_mod(7, 3) = {intro.eval_cpp('_brian_mod(7, 3)', 'int32_t')}")


EVAL C++
M_PI = 3.141592653589793
sizeof(double) = 8
_brian_mod(7, 3) = 1


In [11]:
# --- Function body replacement ---
body = intro.get_body("*stateupdater*", "run")
body

'\n    \n    // scalar code (runs once, outside the loop)\n    const size_t _vectorisation_idx = -1;\n        \n    const double dt = _ptr_array_defaultclock_dt[0];\n    const double t = _ptr_array_defaultclock_t[0];\n    const int64_t _lio_1 = _timestep(0.005, dt);\n    const double _lio_2 = 1.0f*(- dt)/tau;\n    const double _lio_3 = exp(_lio_2);\n\n\n    const int _N = N;\n\n    // vector code (runs per neuron)\n    for (int _idx = 0; _idx < _N; _idx++) {\n        const size_t _vectorisation_idx = _idx;\n                \n        const double lastspike = _ptr_array_neurongroup_lastspike[_idx];\n        char not_refractory = _ptr_array_neurongroup_not_refractory[_idx];\n        double v = _ptr_array_neurongroup_v[_idx];\n        const double v0 = _ptr_array_neurongroup_v0[_idx];\n        not_refractory = _timestep(t - lastspike, dt) >= _lio_1;\n        double _v;\n        if(!not_refractory)\n            _v = (v + v0) - v0;\n        else \n            _v = ((_lio_3 * v) + v0) - (_lio

In [12]:
v_before = np.array(group.v[:])
new_body = body.replace("exp(_lio_2)", "(1.0 + _lio_2)")
print("\n" + "=" * 60)
print("REPLACING WITH LINEAR APPROXIMATION")
print("=" * 60)
versioned_name = intro.replace_body("*stateupdater*", "run", new_body)
versioned_name

INFO       introspector: compiling _brian_cppyy_run_neurongroup_stateupdater_codeobject_v1 (replacing _brian_cppyy_run_neurongroup_stateupdater_codeobject) [brian2.codegen.runtime.cppyy_rt.introspector]



REPLACING WITH LINEAR APPROXIMATION


'_brian_cppyy_run_neurongroup_stateupdater_codeobject_v1'

In [13]:
run(100 * ms)
v_after_mod = np.array(group.v[:])

In [14]:
# --- Restore ---
intro.restore("*stateupdater*", "run")


INFO       introspector: restored original neurongroup_stateupdater_codeobject.run [brian2.codegen.runtime.cppyy_rt.introspector]


In [15]:
intro.inject_cpp("""
inline double leaky_relu(double x, double alpha) {
    return x > 0.0 ? x : alpha * x;
}
""")
print(f"\nleaky_relu(-5, 0.01) = {intro.eval_cpp('leaky_relu(-5.0, 0.01)')}")
print(f"leaky_relu(3, 0.01) = {intro.eval_cpp('leaky_relu(3.0, 0.01)')}")


INFO       introspector: injected custom C++ code [brian2.codegen.runtime.cppyy_rt.introspector]



leaky_relu(-5, 0.01) = -0.05
leaky_relu(3, 0.01) = 3.0


In [16]:
intro.print_objects()

                               Compiled Code Objects                               
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃   ┃ Code Object                                ┃ Template     ┃ Blocks ┃ # Vars ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ ● │ neurongroup_spike_resetter_codeobject      │ reset        │ run    │      5 │
│ ● │ neurongroup_spike_resetter_codeobject_1    │ reset        │ run    │      5 │
│ ● │ neurongroup_spike_thresholder_codeobject   │ threshold    │ run    │      8 │
│ ● │ neurongroup_spike_thresholder_codeobject_1 │ threshold    │ run    │      8 │
│ ● │ neurongroup_stateupdater_codeobject        │ stateupdate  │ run    │     11 │
│ ● │ neurongroup_stateupdater_codeobject_1      │ stateupdate  │ run    │     11 │
│ ● │ statemonitor_codeobject                    │ statemonitor │ run    │      8 │
│ ● │ statemonitor_codeobject_1                  │ statemonitor │ run    │      8 │
└───┴────────────────────────────────────────────┴──────────────┴────────┴────────┘

In [17]:
# --- Snapshot ---
intro.snapshot("*stateupdater*")

{'name': 'neurongroup_stateupdater_codeobject',
 'sources': {'run': '#ifndef _BRIAN_CPPYY_SC_41c7dc13377ec1af\n#define _BRIAN_CPPYY_SC_41c7dc13377ec1af\n// Per-codeobject support code\n\ninline int64_t _timestep(double t, double dt) {\n    return (int64_t)((t + 1e-3*dt)/dt);\n}\n\n// Template-specific support code (e.g. synaptic queue access)\n\n#endif // _BRIAN_CPPYY_SC_41c7dc13377ec1af\n\nextern "C" void _brian_cppyy_run_neurongroup_stateupdater_codeobject(int64_t N, double* _ptr_array_defaultclock_dt, double* _ptr_array_neurongroup_lastspike, int _numlastspike, int8_t* _ptr_array_neurongroup_not_refractory, int _numnot_refractory, double* _ptr_array_defaultclock_t, double tau, double* _ptr_array_neurongroup_v, int _numv, double* _ptr_array_neurongroup_v0, int _numv0) {\n    \n    // scalar code (runs once, outside the loop)\n    const size_t _vectorisation_idx = -1;\n        \n    const double dt = _ptr_array_defaultclock_dt[0];\n    const double t = _ptr_array_defaultclock_t[0];\n 